# Hello World Transformers


In [4]:
from transformers import pipeline
import torch

# Check if we can use GPU (MPS for Mac, CUDA for Linux/Windows)
if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print(f"Device set to use: {device}")


Device set to use: mps


## 1. Text Classification

Text classification is one of the most common NLP tasks. Let's start with sentiment analysis.

### 📚 Question 1: Understanding Pipelines

1. What is a pipeline in Hugging Face Transformers? What does it abstract away from the user?
2. Visit the pipeline documentation and list at least 3 other tasks (besides text-classification) that are available.
3. What happens when you don't specify a model in the pipeline? How can you specify a specific model?

### 📚 Question 2: Text Classification Deep Dive

1. What is the default model used for text-classification? Look at the output above to find its name, then search for it on the Hugging Face Model Hub.
2. What dataset was this model fine-tuned on? What kind of text does it work best with?
3. The output includes a score field. What does this score represent? What range of values can it have?
4. Challenge: Find a different text-classification model on the Hub that classifies emotions (not just positive/negative). What is its name?


## **Question 1: Understanding Pipelines**

### **1. What is a pipeline?**
A pipeline is a ready-to-use Hugging Face tool that bundles all NLP steps into one interface (tokenization, model loading, preprocessing, inference, post-processing).

### **What does it abstract away?**
- Tokenization  
- Model loading  
- Preprocessing  
- Inference  
- Post-processing  
All done with:
```python
pipeline("task-name")
2. Other available pipeline tasks
"ner", "question-answering", "summarization", "translation", "text-generation", "zero-shot-classification", "fill-mask"...

3. What if you don’t define a model?
A default model is selected. To specify one:
classifier = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english")
Question 2: Text Classification Deep Dive
1. Default model
distilbert-base-uncased-finetuned-sst-2-english (DistilBERT for sentiment analysis).

2. Dataset
Fine-tuned on SST-2, a movie review sentiment dataset.
Best for short/simple text; struggles with sarcasm or mixed emotions.

3. Score meaning
Confidence from 0.0–1.0; higher is more confident.
Binary scores ≈ sum to 1.0.

4. Emotion classification models
j-hartmann/emotion-english-distilroberta-base



In [6]:
# Text Classification - Sentiment Analysis
classifier = pipeline("text-classification")

text = "I love using Transformers!"
results = classifier(text)

print("Text:", text)
print("Classification:", results)


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0


Text: I love using Transformers!
Classification: [{'label': 'POSITIVE', 'score': 0.9994327425956726}]


In [7]:
# Try with multiple texts
texts = [
    "I'm really excited about learning Transformers!",
    "This is terrible and I hate it.",
    "The weather is okay today."
]

results = classifier(texts)
for text, result in zip(texts, results):
    print(f"Text: {text}")
    print(f"Label: {result['label']}, Score: {result['score']:.4f}\n")


Text: I'm really excited about learning Transformers!
Label: POSITIVE, Score: 0.9997

Text: This is terrible and I hate it.
Label: NEGATIVE, Score: 0.9995

Text: The weather is okay today.
Label: POSITIVE, Score: 0.9998




### 📚 Question 3: Named Entity Recognition (NER)

1. What does the aggregation_strategy="simple" parameter do in the NER pipeline? Check the token classification documentation.
2. Looking at the output above, what do the entity types mean? (ORG, MISC, LOC, PER)
3. Why do some words appear with ## prefix (like ##tron and ##icons)? What does this indicate about tokenization
4. The model seems to have split "Megatron" and "Decepticons" incorrectly. Why might this happen? What does this tell you about the model's training data?




1. The default NER model can recognize these entity types:
   - PER (Person)
   - ORG (Organization)
   - LOC (Location)
   - MISC (Miscellaneous): other entities
   - O (Outside): not an entity

2. BIO tagging is a way to label words:
   - B-XXX means Beginning of an entity (e.g., B-PER for the first word of a person's name)
   - I-XXX means Inside an entity (e.g., I-PER for the rest of a person's name)
   - O means Outside 

3. To extract only specific entities, you can filter the results:
   ``` python
   persons = [e for e in results if e['entity_group'] == 'PER']
   orgs = [e for e in results if e['entity_group'] == 'ORG']
   ```

4. Yes! You can find multilingual models like `xlm-roberta-base-finetuned-conll03-english` or search the Model Hub for models that support a large range of languages


In [8]:
# Named Entity Recognition
ner = pipeline("ner", aggregation_strategy="simple")

text = "Apple Inc. was founded by Steve Jobs in Cupertino, California. The company is now led by Tim Cook."
results = ner(text)

print("Text:", text)
print("\nNamed Entities:")
for entity in results:
    print(f"{entity['word']}: {entity['entity_group']} (confidence: {entity['score']:.4f})")


No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision 4c53496 (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps:0


Text: Apple Inc. was founded by Steve Jobs in Cupertino, California. The company is now led by Tim Cook.

Named Entities:
Apple Inc: ORG (confidence: 0.9996)
Steve Jobs: PER (confidence: 0.9926)
Cupertino: LOC (confidence: 0.9768)
California: LOC (confidence: 0.9988)
Tim Cook: PER (confidence: 0.9997)


### 📚 Question 4: Question Answering Systems

1. What type of question answering is this? (Extractive vs. Generative) Check the question answering documentation.
2. The model outputs start and end indices. What do these represent? Why are they important?
3. What is the SQuAD dataset? (Look up the model distilbert-base-cased-distilled-squad on the Hub)
4. What happens if the answer is not in the context?




1. Extractive Q1: Finds and copies the exact answer from the context text. Like highlighting a sentence from a book and Generative QA ->  Creates a new answer, even if it's not written exactly that way in the context. Like writing an answer in your own words.

2. The default QA model uses BERT architecture. BERT is good at understanding relationships between words.

3. The model looks at every word in the context and decides which word is most likely the start of the answer, which word is most likely the end of the answer, it finds the beginning and end with the highest confidence score.

4. If the answer isn't in the context, the model will still try to find something, but the confidence score will be low.


In [9]:
# Question Answering
qa = pipeline("question-answering")

context = """
Transformers are a type of neural network architecture introduced in the paper "Attention is All You Need" 
by Vaswani et al. in 2017. They revolutionized natural language processing by using self-attention mechanisms 
instead of recurrent layers. The Hugging Face library makes it easy to use pre-trained transformer models 
for various NLP tasks.
"""

question = "What year were Transformers introduced?"
result = qa(question=question, context=context)

print("Question:", question)
print("Answer:", result['answer'])
print("Confidence:", result['score'])
print(f"Answer span: {result['start']}-{result['end']}")


No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0


Question: What year were Transformers introduced?
Answer: 2017
Confidence: 0.989996075630188
Answer span: 130-134


In [10]:
# Multiple questions
questions = [
    "What paper introduced Transformers?",
    "What mechanism do Transformers use instead of recurrent layers?",
    "What library makes it easy to use transformers?"
]

for question in questions:
    result = qa(question=question, context=context)
    print(f"Q: {question}")
    print(f"A: {result['answer']} (confidence: {result['score']:.4f})\n")


Q: What paper introduced Transformers?
A: Attention is All You Need (confidence: 0.6045)

Q: What mechanism do Transformers use instead of recurrent layers?
A: self-attention (confidence: 0.5299)

Q: What library makes it easy to use transformers?
A: Hugging Face (confidence: 0.3635)




### 📚 Question 5: Text Summarization

1. What is the difference between extractive and abstractive summarization? Check the summarization documentation.
2. Looking at the code in the next cell, what is the default model used for summarization? Search for it on the Hugging Face Model Hub and determine:Is it an extractive or abstractive model? What architecture does it use? (Hint: look at the model name) What dataset was it trained on?
3. What do the max_length and min_length parameters control? What happens if min_length > max_length?
4. The parameter clean_up_tokenization_spaces=True is used. What does this parameter do? Why might it be useful for summarization?




### ANSWERS
1. Extractive summarization picks important sentences directly from the original text and puts them together. Like copying key paragraphs whereas abstractive summarization creates new sentences that capture the main ideas. Like writing a summary in your own words

2. The default summarization model uses BART or T5 architecture. These are good at generating new text while keeping the main meaning.

3. max_length is the maximum number of words/tokens the summary can have. Longer summaries have more detail and min_length is the minimum number of words/tokens the summary must have. Ensures the summary isn't too short.

4. You can find multilingual summarization models on the Model Hub. Models like mBART or mT5 can work with multiple languages

In [11]:
# Text Summarization
summarizer = pipeline("summarization")

text = """
Artificial intelligence (AI) has revolutionized numerous industries, from healthcare to finance, 
transportation to entertainment. Machine learning, a subset of AI, enables computers to learn 
from data without being explicitly programmed. Deep learning, in turn, is a subset of machine 
learning that uses neural networks with multiple layers to model and understand complex patterns.

Natural Language Processing (NLP) is one of the most exciting applications of AI, allowing machines 
to understand, interpret, and generate human language. Recent advances in transformer architectures 
have led to breakthroughs in machine translation, text generation, and language understanding.

These technologies are now being integrated into everyday applications, making our interactions 
with computers more natural and intuitive. However, as AI becomes more powerful, it's crucial to 
consider ethical implications and ensure these systems are used responsibly.
"""

summary = summarizer(text, max_length=100, min_length=50, do_sample=False)
print("Original text length:", len(text), "characters")
print("\nSummary:")
print(summary[0]['summary_text'])


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0


Original text length: 958 characters

Summary:
 Artificial intelligence (AI) has revolutionized numerous industries, from healthcare to finance . As AI becomes more powerful, it's crucial to consider ethical implications and ensure these systems are used responsibly . Natural Language Processing (NLP) is one of the most exciting applications of AI, allowing machines to understand, interpret, and generate human language .



### 📚 Question 6: Machine Translation

1. What is the architecture behind the Helsinki-NLP/opus-mt-en-de model? Look it up on the Model Hub. What does "OPUS" stand for? What does "MT" stand for?
2. How would you find a model to translate from English to French? Visit the translation documentation and the Model Hub to find at least 2 different models.
3. What is the difference between bilingual and multilingual translation models? What are the advantages and disadvantages of each?
4. In the code, we specify the task as "translation_en_to_de". How does this relate to the model we're loading?
5. The output shows a warning about sacremoses. What is this library used for in NLP? Check the MarianMT documentation.
6. Challenge: Find a multilingual model (like mBART or M2M100) that can translate between multiple language pairs. How many language pairs does it support?


### ANSWERS
1. OPUS refers to the dataset/collection whereas OPUS refers to a large set of parallel (translated) texts. MT means Machine Translation. So OPUS-MT = machine translation trained on data from OPUS. 
2. Helsinki-NLP/opus-mt-en-fr => English to French translation model from OPUS-MT, Helsinki-NLP/opus-mt-tc-big-en-fr => another English→French model (a “big” variant) in the same project.
3. 
    - Bilingual model: trained for a single language pair (e.g. English → French) => Pros : often higher translation quality for that pair, simpler, less interference, Cons => you need one model per language pair; many models required to cover many languages.
    - Multilingual model: trained to handle many languages and many translation directions with a single model Pros: one model covers many languages, easier to handle many-to-many translation, shares information across languages, Cons: performance may be lower compared to specialized bilingual for a given pair; may be larger and heavier; risk of “interference” between languages.

4. "translation_en_to_de" tells the pipeline that the source language is English and the target is German. If you load a model like opus-mt-en-de, it matches that task so translation works as expected.
5. sacremoses is used to split text into tokens or to detokenize text (merge tokens back to readable text). 
6. mBART-50 supports 50 languages, meaning it can translate between any pair of these, it yields up to 50×49= 2450 directed language pairs

In [12]:
# Translation - English to German
text = """Dear Amazon, last week I ordered an Optimus Prime action figure from your online store in Germany. 
Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure 
of Megatron instead! As a lifelong enemy of the Decepticons, I hope you can understand my dilemma. 
To resolve the issue, I demand an exchange of Megatron for the Optimus Prime figure I ordered. 
Enclosed are copies of my records concerning this purchase. I expect to hear from you soon. 
Sincerely, Bumblebee."""

translator = pipeline("translation_en_to_de", 
                      model="Helsinki-NLP/opus-mt-en-de")
outputs = translator(text, clean_up_tokenization_spaces=True, min_length=100)
print("Original (English):")
print(text)
print("\n" + "="*80 + "\n")
print("Translation (German):")
print(outputs[0]['translation_text'])


Device set to use mps:0


Original (English):
Dear Amazon, last week I ordered an Optimus Prime action figure from your online store in Germany. 
Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure 
of Megatron instead! As a lifelong enemy of the Decepticons, I hope you can understand my dilemma. 
To resolve the issue, I demand an exchange of Megatron for the Optimus Prime figure I ordered. 
Enclosed are copies of my records concerning this purchase. I expect to hear from you soon. 
Sincerely, Bumblebee.


Translation (German):
Sehr geehrter Amazon, letzte Woche habe ich eine Optimus Prime Action Figur aus Ihrem Online-Shop in Deutschland bestellt. Leider, als ich das Paket öffnete, entdeckte ich zu meinem Entsetzen, dass ich stattdessen eine Action Figur von Megatron geschickt worden war! Als lebenslanger Feind der Decepticons, Ich hoffe, Sie können mein Dilemma verstehen. Um das Problem zu lösen, Ich fordere einen Austausch von Megatron für die Optimus Pri

In [ ]:
# Try English to French
translator_fr = pipeline("translation_en_to_fr", 
                         model="Helsinki-NLP/opus-mt-en-fr")

short_text = "Hello, how are you today? I hope you're having a wonderful day."
translation = translator_fr(short_text, clean_up_tokenization_spaces=True)
print("English:", short_text)
print("French:", translation[0]['translation_text'])



### 📚 Question 7: Text Generation

1. What is the default model used for text generation in the code below? Look it up on the Hub and answer:
   - What architecture does GPT-2 use? (decoder-only, encoder-decoder, or encoder-only?)
   - How many parameters does the base GPT-2 model have?
   - What type of generation does it perform? (autoregressive, non-autoregressive, etc.)
2. Why do we use `set_seed(42)` before generation? What would happen without it?
3. The code uses `max_length=200`. What other parameters can control text generation? Research and explain:
   - `temperature`
   - `top_k`
   - `do_sample`
4. Looking at the output, you can see a warning about truncation. What does this mean? Why is the input being truncated?
5. What does `pad_token_id` being set to `eos_token_id` mean? Why is this necessary for GPT-2?
6. What are the trade-offs between model size and generation quality?


### Answers



### 1. The default model is GPT-2

- Architecture: Decoder only 
- Number of parameters: approx. 124 million 
- Type of generation: Autoregressive



### 2. It makes the output repeatable.  Without it, every time you run the code, the generated text would be different.



### 3.
- temperature: Controls creativity.  
  - Low = safer,more predictable  
  - High = more random, more creative  
- top_k: Keeps only the*top k most likely words
  - Smaller k = safer  
  - Larger k =more variety  
- do_sample: 
  - If True => model samples words
  - If False => model picks the most likely word each time


### 4. The model cuts the input because it is too long  and exceeds GPT 2’s maximum token limit.  So the model uses only the allowed number of tokens.


### 5. GPT-2 has no pad token , so we reuse the end-of-sentence token as padding.  This avoids errors during generation.


### 6. 
- Small models:
  - Faster, lighter  
  - Lower quality, more mistakes  
- Large models: 
  - Slower, require more memory  
  - Much better, more coherent text  




In [13]:
from transformers import set_seed
set_seed(42)  # Set the seed to get reproducible results


In [14]:
# Text Generation
generator = pipeline("text-generation")

response = "Dear Bumblebee, I am sorry to hear that your order was mixed up."
prompt = text + "\n\nCustomer service response:\n" + response

outputs = generator(prompt, max_length=200, truncation=True)
print(outputs[0]['generated_text'])


No model was supplied, defaulted to openai-community/gpt2 and revision 607a30d (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Dear Amazon, last week I ordered an Optimus Prime action figure from your online store in Germany. 
Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure 
of Megatron instead! As a lifelong enemy of the Decepticons, I hope you can understand my dilemma. 
To resolve the issue, I demand an exchange of Megatron for the Optimus Prime figure I ordered. 
Enclosed are copies of my records concerning this purchase. I expect to hear from you soon. 
Sincerely, Bumblebee.

Customer service response:
Dear Bumblebee, I am sorry to hear that your order was mixed up. I did purchase the right size Optimus Prime figure for me, but I did not receive the correct size shipment. I appreciate your understanding as to the issue.

Sincerely, Optimus Prime.

Customer service response:

Dear Optimus Prime, I have been shopping and seeing a lot of Transformers online. 

This issue is one of my favorite things about the Transformers series. 

However, I can find

In [15]:
# Experiment with different generation parameters
prompt = "The future of artificial intelligence"

# Different temperature settings
print("="*80)
print("Temperature = 0.7 (more focused):")
output1 = generator(prompt, max_length=100, temperature=0.7, do_sample=True, truncation=True)
print(output1[0]['generated_text'])

print("\n" + "="*80)
print("Temperature = 1.5 (more creative):")
output2 = generator(prompt, max_length=100, temperature=1.5, do_sample=True, truncation=True)
print(output2[0]['generated_text'])


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Temperature = 0.7 (more focused):


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=100) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


The future of artificial intelligence is at stake. It is not a matter of whether this will happen in the near future, or when, but a question of how we can get there.

The answer is that we should be able to build robots capable of doing things we do not have the means to do. Our ability to do so is largely driven by our ability to design new and more powerful machines that are capable of doing these sorts of things. We are all just now discovering that, even if we are able to do everything we want, there are limits to our ability to do them.

Advertisement

We might want to build robots that are capable of doing things that we don't have the means to do. But what about a computer that can do things we do not have the means to do?

That is a much broader question than is often asked, and this raises many different questions.

What do we do with AI, and how do we create an artificial intelligence that can do things that we do not have the means to do? What do we do with the power of art